# Data Analysis
### An exploratory data analysis of our collected data.

### Objectives

1. Answer our first Business requirement. 
    * Perform a correlation and/or PPS study to investigate the most relevant variables correlated to the sale price.
    * Visualize these variables against the sale price, in order to summarize the insights.

### Inputs

1. Our house_price_records data that we collected in our DataCollection Notebook, found at inputs/datasets/raw/house-price-20211124T154130z/house-price/house_price_records.csv 

### Outputs

1. Code that succesfully generates the answers to our first business requirement.
2. Plotted graphs that visualise the results found during correlation testing.
3. Useful and insightful analysis of the found results.

### Additional Comments

* This notebook was designed and follows the principles set out by Code Institute in the Predictive Analytics lessons and Walkthrough projects. The code written in this work book has taken influence from these lessons and projects but has been modiefied, in some cases such as the graphical design, heavily modified by myself in order to suit the needs for this project.

___

## Change working Directory

* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir


We want to make the parent of the current directory the new current directory

* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

## Load the data

The first step is simply to load in our raw data that we collected in the previous notebook.

In [ ]:
import pandas as pd
df = (pd.read_csv("inputs/datasets/raw/house-price-20211124T154130Z-001/house-price/house_prices_records.csv"))
df.head()

## Data Exploration

* With our data loaded we can begin to analyse it. 
    * To begin with I have generated a panadas profile report, this is a an automated dashboard used for quick EDA (Exploratory Data Analysis), simply run the code below to genorate the report.

In [ ]:
from pandas_profiling import ProfileReport
pandas_report = ProfileReport(df=df, minimal=True)
pandas_report.to_notebook_iframe()

The Pandas report we have generated provides a lot of very useful insights, we can see straight away a summary of each variable in our data. This summary details information such as the mean, min and max values, the number of missing values if any, and a graph showing the distribution of values.

* An instant take away from this report is the amount of missing values, if we have a look at the alerts tab we can see that 8 columns contain missing values. Missing values appear in both numerical and categorical variables, there are various different avenues we can take to deal with missing values. These will be reviewed in Data Cleaning.

___

## Correlation Study

1. Firstly in order to assess the correlation between Sales price and any numerical variables in our dataset I will use Spearman and Pearson correlation tests on our raw data.
    * Pearson correlation assess the linear relationship between two variables and produces a value between -1 and +1, donating a negative or positive correlation respectivly. a value of 0 indictaes no corraelation between the variables.
    * Spearman correlation assess the monotonic relation between vairables. Again it provides a value between -1 and +1.

2. These two tests only evaluate the correlation of numerical values. Therefore we are not getting any correlation information regarding our categorical variables such as Kitchen Quality. 
    * In order to evaluate these variables in conjucture with the rest I will perform some simple data cleaning and feature engineering.
    * The data will have the null values amended, categorical variables will have 'missing' in place of null values, and the mean will be added in place of any null values in the numerical data.
    * One hot encoder will then be used to convert the categorical variables to a binary classification allowing us to evaluate them using spearman and pearson correlation tests.
3. The results of these 4 tests will then be evaluated and the leading variables that show the greatest correlation will be plotted.   

In [ ]:
corr_spearman = df.corr(method='spearman')['SalePrice'].sort_values(key=abs, ascending=False)[1:].head(10)
corr_spearman

In [ ]:
corr_pearson = df.corr(method='pearson')['SalePrice'].sort_values(key=abs, ascending=False)[1:].head(10)
corr_pearson

In [ ]:
from sklearn.pipeline import Pipeline
from feature_engine.encoding import OneHotEncoder
from feature_engine.imputation import DropMissingData, MeanMedianImputer, CategoricalImputer

pipeline = Pipeline([
    ('mmi', MeanMedianImputer(imputation_method='mean', variables=['2ndFlrSF','EnclosedPorch','GarageYrBlt','LotFrontage','WoodDeckSF'])),
    ('ci', CategoricalImputer(imputation_method='missing', variables=['BsmtExposure','BsmtFinType1','GarageFinish','KitchenQual'])),
    ('ohe', OneHotEncoder(variables=['BsmtExposure','BsmtFinType1','GarageFinish','KitchenQual'], drop_last=False))
])
df_ohe = pipeline.fit_transform(df)
print(df_ohe.shape)
df_ohe.head(300)

In [ ]:
corr_spearman_ohe = df_ohe.corr(method='spearman')['SalePrice'].sort_values(key=abs, ascending=False)[1:].head(10)
corr_spearman_ohe

In [ ]:
corr_pearson_ohe = df_ohe.corr(method='pearson')['SalePrice'].sort_values(key=abs, ascending=False)[1:].head(10)
corr_pearson_ohe

Upon completion of our correlation tests, we can take the top 6 correlation levels for both sets of tests to review if any differences have arised.

In [ ]:
top_n = 5
set(corr_pearson[:top_n].index.to_list() + corr_spearman[:top_n].index.to_list())

In [ ]:

set(corr_pearson_ohe[:top_n].index.to_list() + corr_spearman_ohe[:top_n].index.to_list())

* We can see that both sets of tests have produced the same top 6 most correlated variables. The findings are as follows:
    1. A house with a high sales price typically has a large 1st floor square footage.
    2. A house with a high sales price typically has a large garage square footage.
    3. A house with a high sales price typically has a large above grade (ground) living area square footage.
    4. A house with a high sales price typically has a high overall quality.
    5. A house with a high sales price typically has a large basement square footage.
    6. A house with a high sales price typically was built more recently.

I will use the raw data (df) to plot graphs of the target variables. 

In [ ]:
target_variables = ['1stFlrSF', 'GarageArea', 'GrLivArea', 'OverallQual', 'TotalBsmtSF', 'YearBuilt']
target_variables

___

## EDA on target_variables

In [ ]:
df_eda = df.filter(target_variables + ['SalePrice'])
df_eda.head(5)

In [ ]:
import plotly.express as px


def plot_numerical(df, col, target_var):
    fig = px.scatter(data_frame=df_eda, x=col, y=target_var, marginal_x='box', trendline='ols', trendline_color_override='red' ,title=f'{col} against {target_var}',width=1400, height=800)
    fig.show()


target_var = 'SalePrice'
for col in target_variables:
    plot_numerical(df_eda, col, target_var)
    print("\n\n")

___

## Conclusions on Data Analysis

* To review, the 6 variables that were highligthed and our assumptions made on them are as follows:
    1. A house with a high sales price typically has a large 1st floor square footage.
    2. A house with a high sales price typically has a large garage square footage.
    3. A house with a high sales price typically has a large above grade (ground) living area square footage.
    4. A house with a high sales price typically has a high overall quality.
    5. A house with a high sales price typically has a large basement square footage.
    6. A house with a high sales price typically was built more recently.


* Upon further analysis of these variables through plotting we can confirm that each variable does indeed have a strong postive correlation to the sales price as indicated by our postive trendlines. Therefore we can assume they may, be well suited for predicting the future sales price of an unseen house.
    * It is clear through our analysis that some of our identified varibales look to have a greater impact then others, for example the GrLivArea and TotalBsmtSF look to have a greater influence over sales price then the year built. However all variables look to have an impact so all will be considered during our modelling.

* The next step is to clean the data prior to feature engineering.

___